In [1]:
import urllib.request
import json
import spotipy
import webbrowser
import spotipy.util as util

In [2]:
credentials = "spotify_keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [3]:
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]

In [4]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-read-recently-played'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [5]:
sp = spotipy.Spotify(auth=token)

In [6]:
%run Weather.ipynb
%run OuraRing.ipynb
%run News.ipynb

In [7]:
# user inputs their start station
station_start_input = input('Enter from station')
station_end_input = input('Enter to station')

# remove spaces from user input
input_start_ns = station_start_input.replace(" ", "")
input_end_ns = station_end_input.replace(" ", "")


def get_station_id_from_user_input(station):

    try:
        url = (f"https://api.tfl.gov.uk/StopPoint/Search?query={station}")

        hdr ={
        # Request headers
        'Cache-Control': 'no-cache',
        }

        req = urllib.request.Request(url, headers=hdr)

        req.get_method = lambda: 'GET'
        response = urllib.request.urlopen(req)
        station_id = response.read()
        station_id_decoded = station_id.decode('utf-8')
        station_json = json.loads(station_id_decoded)
        id = station_json['matches'][0]['id']
        return id
    except Exception as e:
        return print(e)


start_station = get_station_id_from_user_input(input_start_ns)
end_station = get_station_id_from_user_input(input_end_ns)

print(start_station)
print(end_station)


Enter from station Canada Water Underground Station
Enter to station Farringdon Underground Station


940GZZLUCWR
940GZZLUFCN


In [11]:
# use station start id to find crowd level (percentage of baseline)

def find_crowd_levels(station_id):
    try:
        url = (f"https://api.tfl.gov.uk/crowding/{station_id}/Live")

        hdr ={
        # Request headers
        'Cache-Control': 'no-cache',
        }

        req = urllib.request.Request(url, headers=hdr)

        req.get_method = lambda: 'GET'
        response = urllib.request.urlopen(req)
        station_data = response.read()
        station_data_decoded = station_data.decode('utf-8')
        station_data_decoded_json = json.loads(station_data_decoded)
        percentage_of_baseline = station_data_decoded_json["percentageOfBaseline"]
        return percentage_of_baseline
    except Exception as e:
        print(e)

crowding_start = find_crowd_levels(start_station)
crowding_end = find_crowd_levels(end_station)


In [12]:
average_crowd = round(((crowding_start + crowding_end)/2),2)

In [13]:
tfl_min = 0
tfl_max = 0

if average_crowd < 0.2:
    tfl_min = 0
    tfl_max = 50
    
elif average_crowd > 0.2 and average_crowd < 0.4:
    tfl_min = 50
    tfl_max = 100
    
elif average_crowd > 0.4 and average_crowd < 0.6:
    tfl_min = 100
    tfl_max = 150
    
elif average_crowd > 0.6 and average_crowd < 0.8:
    tfl_min = 150
    tfl_max = 200
    
elif average_crowd > 0.8:
    tfl_min = 200
    tfl_max = 250

In [64]:
recently_played = sp.current_user_recently_played(limit=1,after=None, before=None)
song = recently_played['items']
artist_id = [song[0]['track']['artists'][0]['id']]
track_id = [song[0]['track']['id']]

HTTP Error for GET to https://api.spotify.com/v1/me/player/recently-played with Params: {'limit': 1, 'after': None, 'before': None} returned 403 due to User not registered in the Developer Dashboard


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/me/player/recently-played?limit=1:
 User not registered in the Developer Dashboard, reason: None

In [65]:
tracks = sp.recommendations(artist_id,track_id, target_danceability=fnews_score, target_energy=oura, target_valence=weather_mood, min_tempo=tfl_min, max_tempo= tfl_max, genres=None, limit=15)
track_list = tracks['tracks']
uri_list = []
for track in track_list:
    uri_list.append(track['uri'])

NameError: name 'artist_id' is not defined

In [66]:
my_playlist = sp.user_playlist_create(user=username, name="today", public=True, description="Songs for today")
a = sp.user_playlist_add_tracks(username, my_playlist['id'], uri_list)

HTTP Error for POST to https://api.spotify.com/v1/users/jessiealiang/playlists with Params: {} returned 403 due to User not registered in the Developer Dashboard


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/users/jessiealiang/playlists:
 User not registered in the Developer Dashboard, reason: None

In [67]:
webbrowser.open(my_playlist['external_urls']['spotify'])

NameError: name 'my_playlist' is not defined